This is the workflow for ASE-phonopy interface!
Run the code below and you will get some phonopy result!


In [ ]:
!rm -r EMT

In [ ]:
#Build primitive cell(unit cell) in diffenet LC
from ase.io import read,write
import sys, os
from pathlib import Path
from ase.lattice.cubic import FaceCenteredCubic
from ase.build import bulk
import numpy as np
import copy
metal='Pt'

currentpath=os.getcwd()
#os.chdir(currentpath)
Oripath=copy.copy(os.getcwd())#redirection
LatticeConstant=np.arange(3.89,3.96,0.01)
for LC in LatticeConstant:
        LCfolder=os.path.join('EMT',str(LC))#path of the LC relaxation
        Path(LCfolder).mkdir(parents=True, exist_ok=True)
        ##If you want to build the conventinal cell use the code below:
        #atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
        #      symbol='Pt',
        #      size=(1, 1, 1),
        #      pbc=True,latticeconstant=LC)
        atoms = bulk('Pt', 'fcc', a=LC) #primitive cell of Pt
        atoms.write(LCfolder+'/POSCAR',format='vasp',sort=True)# we write it as POSCAR format, so you can also use it to do vasp calcution.
        lc=atoms.get_cell_lengths_and_angles()[0]
        f=open(LCfolder+'/out.LC','w')#Write out the lattice constant
        f.write(str(lc)+'\n')
        f.close()
os.chdir(Oripath)

In [ ]:
##Generate Displacement and related file by phonopy
import os,sys
import copy
import spglib
from pathlib import Path
from phonopy import Phonopy

currentpath=os.getcwd()
Oripath=copy.copy(os.getcwd())#redirection
LatticeConstant=np.arange(3.89,3.96,0.01)
for k,lc in enumerate(LatticeConstant):
    os.chdir(Oripath+'/EMT/'+str(lc))
    print(os.getcwd())
    #n='%02d' %k
    #print(n)
    os.system("phonopy -d --dim='1 1 1'")##generate displacements and super cell
    FolderS=os.path.join(os.getcwd(),'SPOS')#perfect supercell
    #Write a loop here if you have more than one displacement structure
    Folder1=os.path.join(os.getcwd(),'POS001')#displacement cell
    Path(FolderS).mkdir(parents=True, exist_ok=True)
    Path(Folder1).mkdir(parents=True, exist_ok=True)
    os.system('mv SPOSCAR '+FolderS+'/POSCAR')
    os.system('mv POSCAR-001 '+Folder1+'/POSCAR')
os.chdir(Oripath)

In [ ]:
##Calculate and Write out force_set
import os,sys
import copy
import spglib
from pathlib import Path
from phonopy import Phonopy
from ase.io import read,write
from ase.calculators.emt import EMT
import yaml
file_path = "FORCE_SETS"
currentpath=os.getcwd()
os.chdir(currentpath)
Oripath=copy.copy(os.getcwd())#redirection

for k,lc in enumerate(LatticeConstant):
    os.chdir(Oripath+'/EMT/'+str(lc))
    print(os.getcwd())
    atoms1=read('POSCAR')#read unit cell
    len1=len(atoms1)
    Folder1=os.path.join(os.getcwd(),'POS001')
    atoms2=read(Folder1+'/POSCAR')
    #Set the calulater and get forces
    atoms2.calc = EMT() # you can attach to any calculator
    my_force_array = atoms2.get_forces()
    # Load the YAML data from the file
    with open('phonopy_disp.yaml', 'r') as file:
        data = yaml.safe_load(file)
    # Extract the displacement array and atom value
    displacement_array = data['displacements'][0]['displacement']
    atom = data['displacements'][0]['atom']
    with open(file_path, 'w') as file:
        # Iterate over each row in the array
        file.write(str(len(atoms2)) +'\n')
        file.write(str(len(atoms1)) +'\n\n')
        #Write out the displacement 
        file.write(str(atom) +'\n')
        formatted_row = '  '.join('{:18.10f}'.format(d) for d in displacement_array)
        file.write(' ' + formatted_row + '\n')
        for row in my_force_array:
            # Convert each value in the row to the desired format and write to the file
            formatted_row = '  '.join(f"{value:20.16f}" for value in row)
            file.write(' ' + formatted_row + '\n')
os.chdir(Oripath)

In [ ]:
#Calculate the thermal properties 
import os,sys
import copy
import spglib
from phonopy import Phonopy

from ase.io import read,write
import sys, os
from pathlib import Path

from ase.lattice.cubic import FaceCenteredCubic
import yaml

currentpath=os.getcwd()
os.chdir(currentpath)
Oripath=copy.copy(os.getcwd())#redirection
LatticeConstant=np.arange(3.89,3.96,0.01)
for k,lc in enumerate(LatticeConstant):
    os.chdir(Oripath+'/EMT/'+str(lc))
    print(os.getcwd())
    n='%02d' %k
    print(n)
    os.system("phonopy -t -p -s --tmax=2000 --dim='1 1 1' --mp='32 32 32'")#write the parameter of the 
    os.system('cp thermal_properties.yaml '+Oripath+'/thermal_properties.yaml-'+n)
    

In [ ]:
###Get the E-V data and write out
import os,sys
import copy
import spglib
from phonopy import Phonopy
from ase.calculators.emt import EMT

from ase.io import read,write
import sys, os
from pathlib import Path
from ase.lattice.cubic import FaceCenteredCubic
import yaml
import numpy as np

currentpath=os.getcwd()
os.chdir(currentpath)
Oripath=copy.copy(os.getcwd())#redirection
LatticeConstant=np.arange(3.89,3.96,0.01)
dataEV=[]
for k,lc in enumerate(LatticeConstant):
#for k,lc in enumerate(LatticeConstant[0:5]):#if you want to write partly data
    os.chdir(Oripath+'/EMT/'+str(lc))
    print(os.getcwd())
    n='%02d' %k
    print(n)
    Folder1=os.path.join(os.getcwd(),'POS001')
    atoms=read(Folder1+'/POSCAR')
    atoms.calc = EMT()
    dataEV.append([atoms.get_volume(),atoms.get_potential_energy()])

os.chdir(Oripath)
filename = 'e-v.dat'
with open(filename, 'w') as file:
    file.write('#   cell volume        energy of cell other than phonon \n')
    for row in dataEV:
        line = '{:18.8f}  {:18.8f}\n'.format(row[0], row[1])
        file.write(line)

In [ ]:
!rm *.pdf

In [ ]:
##Quasi Harmonic Approximation Output
!phonopy-qha -p -s --tmax=500 e-v.dat thermal_properties.yaml-{00..06}

In [ ]:
import re
import matplotlib.pyplot as plt  # noqa
import numpy as np
# Open the text file and read its contents
with open('helmholtz-volume.dat', 'r') as file:
    data = file.read()

# Define the regular expression patterns
temperature_pattern = r'Temperature:\s([\d.-]+)'
parameter_pattern = r'Parameters:\s([\d.-]+)\s([\d.-]+)\s([\d.-]+)\s([\d.-]+)'

# Find all matches for the temperature pattern
temperatures = re.findall(temperature_pattern, data)

# Find all matches for the parameter pattern
parameters = re.findall(parameter_pattern, data)

# Convert the extracted data to floats
temperatures = [float(temp) for temp in temperatures]
parameter_4 = [(float(param[3])*4)**(1/3)/3 for param in parameters]

# Plot the data
plt.plot(temperatures[::10], parameter_4[::10], 'bo-')
plt.xlabel('Temperature')
plt.ylabel('4th Parameter')
plt.title('Temperature vs 4th Parameter')
plt.grid(True)
plt.show()

"""
# Print the extracted data
for i in range(len(temperatures)):
    temperature = float(temperatures[i])
    parameter_4 = float(parameters[i][3])
    print(f"Temperature: {temperature}, 4th Parameter: {parameter_4}")
"""